In [32]:
import tifffile
import imageio
from cellpose import models
import numpy as np
from cellstitch.pipeline import *
from cellstitch.evaluation import *
import pandas as pd
import h5py

In [55]:
dataset = "Valve"

# Train cellpose model from scratch

In [14]:
for i in range(70): 
    filename = "%s_%02d" % (dataset, i)
    img = np.load("../DATA/%s/%s.npy" % (dataset, filename))
    labels = np.load("../DATA/%s/%s_masks.npy" % (dataset, filename))
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("../DATA/%s/cellpose_train/%s_%s.tif" % (dataset, filename, i), img[i])
        imageio.imwrite("../DATA/%s/cellpose_train/%s_%s_masks.tif" % (dataset, filename, i), labels[i])

`python -m cellpose --train --use_gpu --dir ../DATA/<dataset>/cellpose_train --pretrained_model None --n_epochs 100  --verbose`

###  Generate cellpose3d results

In [56]:
model_dir = '../DATA/Valve/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_05_30_20_25_23.614818'

In [57]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [58]:
for i in range(70, 100): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/%s.npy" % (dataset, test_filename))
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/%s/cellpose3d/%s.npy" % (dataset, test_filename), masks) 

Starting Valve_70
Starting Valve_71
Starting Valve_72
Starting Valve_73
Starting Valve_74
Starting Valve_75
Starting Valve_76
Starting Valve_77
Starting Valve_78
Starting Valve_79
Starting Valve_80
Starting Valve_81
Starting Valve_82
Starting Valve_83
Starting Valve_84
Starting Valve_85
Starting Valve_86
Starting Valve_87
Starting Valve_88
Starting Valve_89
Starting Valve_90
Starting Valve_91
Starting Valve_92
Starting Valve_93
Starting Valve_94
Starting Valve_95
Starting Valve_96
Starting Valve_97
Starting Valve_98
Starting Valve_99


## Generate cellstitch results

In [59]:
for i in range(70, 100): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/%s.npy" % (dataset, test_filename))
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s.npy" % (dataset, test_filename), cellstitch)  

Starting Valve_70
Starting Valve_71
Starting Valve_72
Starting Valve_73
Starting Valve_74
Starting Valve_75
Starting Valve_76
Starting Valve_77
Starting Valve_78
Starting Valve_79
Starting Valve_80
Starting Valve_81
Starting Valve_82
Starting Valve_83
Starting Valve_84
Starting Valve_85
Starting Valve_86
Starting Valve_87
Starting Valve_88
Starting Valve_89
Starting Valve_90
Starting Valve_91
Starting Valve_92
Starting Valve_93
Starting Valve_94
Starting Valve_95
Starting Valve_96
Starting Valve_97
Starting Valve_98
Starting Valve_99


# Benchmark results

In [60]:
ap_threshold = 0.5

In [61]:
methods = ["cellstitch", "cellpose3d"]

for method in methods: 
    data = [] 
    for i in range(70, 100): 
        filename = "%s_%02d" % (dataset, i)
        labels = np.load('../DATA/%s/%s_masks.npy' % (dataset, filename))
        true_num_cells = np.unique(labels).size - 1 
        true_avg_vol = get_avg_vol(labels) 
    
        masks = np.load("./results/%s/%s/%s.npy" % (dataset, method, filename))

        num_cells = np.unique(masks).size - 1
        d_num_cells = abs(num_cells - true_num_cells) / true_num_cells

        avg_vol = get_avg_vol(masks)
        d_avg_vol = abs(true_avg_vol - avg_vol) / true_avg_vol

        ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
    
        row = [
            filename, 
            d_num_cells, 
            d_avg_vol, 
            ap, 
            tp, 
            fp, 
            fn, 
            precision,
            recall, 
            true_num_cells, 
            true_avg_vol, 
            num_cells, 
            avg_vol
        ]

        data.append(row)

    df = pd.DataFrame(data, columns=[
        "filename",  
        "d_num_cells", 
        "d_avg_vol", 
        "ap", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall",
        "true_num_cells", 
        "true_avg_vol", 
        "num_cells", 
        "avg_vol"
    ])

    df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)

/home/yining/anaconda3/envs/segmentation/lib/python3.8/site-packages/cellpose/metrics.py:205: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)
